<a href="https://colab.research.google.com/github/Jatansahu/BIG-DATA-PROCESSING/blob/main/PySpark_Linear_regression_LAB05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IT 609 - Big Data Processing Lab 4
#March 16, 2023

**ID  - 202218061**


**NAME - JATAN SAHU**

#1 Aim
#• learn how to use PySpark for applying Linear Regression.

2 Problem description

Spark provides APIs for various machine learning algorithms. In this lab as-
signment you build Linear regression model which predicts the selling price of

a car based on a set of features like car name,year, km driven, number of seats,
etc.

3 Implementation

3.1 Dataset
• You will use the Vehicle dataset from cardekho Car details v3.csv.

3.2 Exercise

RDD Tasks

• Read the dataset into a PySpark Dataframe.

• You are supposed to use all the columns except for torque (bonus task)
for predicting the selling price using Linear regression.

• Use appropriate functions to transform the column data into numerical
values and handle missing values.

• Compare the results obtained from standardization and without standard-
ization.

• Split the data into train and test (80:20). Train the linear regression model
using train data and evaluate the model on test data.

• Compute RMSE, MAE and R2 score for the test data.

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=9eaff4096da68bb74dd0444023e53efddca6e2eec8173925f4c90c6d4701e769
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Applying Linear regression').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

#Data Pre-Processing

In [ ]:
# Read the dataset into a PySpark Dataframe
df = spark.read.csv("/content/drive/MyDrive/BDP/datasets/Car_details_LAB5.csv",header=True, inferSchema=True )


In [ ]:
df.show()

+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+--------------------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|              torque|seats|
+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+--------------------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|      190Nm@ 2000rpm|    5|
|Skoda Rapid 1.5 T...|2014|       370000|   120000|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp| 250Nm@ 1500-2500rpm|    5|
|Honda City 2017-2...|2006|       158000|   140000|Petrol| Individual|      Manual| Third Owner| 17.7 kmpl|1497 CC|    78 bhp|12.7@ 2,700(kgm@ ...|    5|
|Hyundai i20 Sport...|2010|       225000|   127000|Diesel| Individual|      

In [ ]:
#Shape of dataframe
print((df.count(), len(df.columns)))

(8128, 13)


#You are supposed to use all the columns except for torque (bonus task) for predicting the selling price using Linear regression

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df =df.drop("torque")

In [ ]:
df.show(5)

+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|
+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|
|Skoda Rapid 1.5 T...|2014|       370000|   120000|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|
|Honda City 2017-2...|2006|       158000|   140000|Petrol| Individual|      Manual| Third Owner| 17.7 kmpl|1497 CC|    78 bhp|    5|
|Hyundai i20 Sport...|2010|       225000|   127000|Diesel| Individual|      Manual| First Owner| 23.0 kmpl|1396 CC|    90 bhp|    5|
|Maruti Swift VXI ...|2007|       130000|   120000|Petrol| Individual

In [ ]:
#Different way to drop
# """ import col is required """  
# df.drop(col("torque")) \
#   .printSchema()  
  
# df.drop(df.torque) \
#   .printSchema() 

#• Use appropriate functions to transform the column data into numerical values and handle missing values

#Handling missing values

In [ ]:
dict_null = { col : df.filter(df[col].isNull()).count() for col in df.columns}
dict_null

{'name': 0,
 'year': 0,
 'selling_price': 0,
 'km_driven': 0,
 'fuel': 0,
 'seller_type': 0,
 'transmission': 0,
 'owner': 0,
 'mileage': 221,
 'engine': 221,
 'max_power': 215,
 'seats': 221}

In [ ]:
#Dropping Null values
df =df.na.drop()

In [ ]:
dict_null = { col : df.filter(df[col].isNull()).count() for col in df.columns}
dict_null

{'name': 0,
 'year': 0,
 'selling_price': 0,
 'km_driven': 0,
 'fuel': 0,
 'seller_type': 0,
 'transmission': 0,
 'owner': 0,
 'mileage': 0,
 'engine': 0,
 'max_power': 0,
 'seats': 0}

In [ ]:
#Shape of dataframe
print((df.count(), len(df.columns)))

(7907, 12)


In [ ]:
#Dropping one null value in max_power
df.filter(df['max_power'].isNull()).count()

0

In [ ]:
#dropping null values on basis of a column
df =df.na.drop(subset=["max_power"])
df.filter(df['max_power'].isNull()).count()

0

#Transforming the column data

#Encoding all the string data

In [ ]:
df.show(2)

+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|
+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|
|Skoda Rapid 1.5 T...|2014|       370000|   120000|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|
+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+-----+
only showing top 2 rows



In [ ]:
print(df.select("name").distinct().show())
print(df.select("fuel").distinct().show())
print(df.select("seller_type").distinct().show())
print(df.select("transmission").distinct().show())
print(df.select("owner").distinct().show())

+--------------------+
|                name|
+--------------------+
|  Maruti 800 DX BSII|
| Honda City 1.5 S MT|
|Mahindra Bolero 2...|
|Toyota Etios Liva GD|
|Tata Indica V2 Tu...|
|Mahindra KUV 100 ...|
|Mahindra Xylo D2 ...|
|Datsun GO Anniver...|
|Mahindra Bolero S...|
|Toyota Innova 2.5...|
| Mahindra TUV 300 T4|
|    Datsun GO Plus A|
|Tata Indica V2 DL...|
|Maruti Ciaz 1.4 A...|
|Mahindra Scorpio ...|
|Hyundai i20 1.4 Asta|
|Maruti Swift Lxi ...|
|  Maruti Zen Classic|
|      Maruti Zen LXI|
|Hyundai Verna 1.6...|
+--------------------+
only showing top 20 rows

None
+------+
|  fuel|
+------+
|Diesel|
|   CNG|
|   LPG|
|Petrol|
+------+

None
+----------------+
|     seller_type|
+----------------+
|      Individual|
|          Dealer|
|Trustmark Dealer|
+----------------+

None
+------------+
|transmission|
+------------+
|   Automatic|
|      Manual|
+------------+

None
+--------------------+
|               owner|
+--------------------+
|         Third Owner|
|Fourth & Ab

#Preprocessing

In [ ]:
df.show(1)

+--------------------+----+-------------+---------+------+-----------+------------+-----------+---------+-------+---------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|      owner|  mileage| engine|max_power|seats|
+--------------------+----+-------------+---------+------+-----------+------------+-----------+---------+-------+---------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual|First Owner|23.4 kmpl|1248 CC|   74 bhp|    5|
+--------------------+----+-------------+---------+------+-----------+------------+-----------+---------+-------+---------+-----+
only showing top 1 row



In [ ]:
#Removing substring using the regexp_replace method
from pyspark.sql import functions as F
df = df.withColumn("mileage", F.regexp_replace("mileage", "kmpl", ""))
df = df.withColumn("mileage", F.regexp_replace("mileage", "km/kg", ""))
df = df.withColumn("engine", F.regexp_replace("engine", "CC", ""))
df = df.withColumn("max_power", F.regexp_replace("max_power", "bhp", ""))
df.show()

+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|
+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner|  23.4 | 1248 |      74 |    5|
|Skoda Rapid 1.5 T...|2014|       370000|   120000|Diesel| Individual|      Manual|Second Owner| 21.14 | 1498 |  103.52 |    5|
|Honda City 2017-2...|2006|       158000|   140000|Petrol| Individual|      Manual| Third Owner|  17.7 | 1497 |      78 |    5|
|Hyundai i20 Sport...|2010|       225000|   127000|Diesel| Individual|      Manual| First Owner|  23.0 | 1396 |      90 |    5|
|Maruti Swift VXI ...|2007|       130000|   120000|Petrol| Individual|      Manual| First Owner|  16.1 |

In [ ]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: string (nullable = true)
 |-- engine: string (nullable = true)
 |-- max_power: string (nullable = true)
 |-- seats: integer (nullable = true)



In [ ]:
# Cast mileage from string type to float type , engine from string to integer and max_power string to float
df = df.withColumn("mileage", df["mileage"].cast('float'))
df = df.withColumn("engine", df["engine"].cast('integer'))
df = df.withColumn("max_power", df["max_power"].cast('float'))


In [ ]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: float (nullable = true)
 |-- engine: integer (nullable = true)
 |-- max_power: float (nullable = true)
 |-- seats: integer (nullable = true)



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder
#StringIndexer: This is essentially assigning a numeric value to each category (i.e.; Fair: 0, Ideal: 1, Good: 2, Very Good: 3, Premium: 4)
#OneHotEncoder: This converts categories into binary vectors. The result is a SparseVector that indicates which index from StringIndexer has the one-hot value of 1.

 
cat_cols= ["name", "fuel", "seller_type","transmission","owner"]
stages = [] # Stages in Pipeline

for c in cat_cols:
    stringIndexer = StringIndexer(inputCol=c, outputCol=c + "_index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], \
            outputCols=[c + "_vec"])    
    stages += [stringIndexer, encoder] # Stages will be run later on

#we need to combine all of our features into one vector column.

#combining numerical features and the categorical sparse vector features from before into one vector.

#runing the stages as a pipeline

In [ ]:
df.show(1)

+--------------------+----+-------------+---------+------+-----------+------------+-----------+-------+------+---------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|      owner|mileage|engine|max_power|seats|
+--------------------+----+-------------+---------+------+-----------+------------+-----------+-------+------+---------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual|First Owner|   23.4|  1248|     74.0|    5|
+--------------------+----+-------------+---------+------+-----------+------------+-----------+-------+------+---------+-----+
only showing top 1 row



In [ ]:
from pyspark.ml.feature import VectorAssembler

# Transform all features into a vector
num_cols = ["year", "km_driven", "mileage","engine", "seats"]
assemblerInputs = [c + "_vec" for c in cat_cols] + num_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

# Create pipeline and use on dataset
pipeline = Pipeline(stages=stages)
df = pipeline.fit(df).transform(df)

In [ ]:
df.show(2)

+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+----------+------------------+----------+-------------+-----------------+---------------+------------------+----------------+-----------+-------------+--------------------+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|name_index|          name_vec|fuel_index|     fuel_vec|seller_type_index|seller_type_vec|transmission_index|transmission_vec|owner_index|    owner_vec|            features|
+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+----------+------------------+----------+-------------+-----------------+---------------+------------------+----------------+-----------+-------------+--------------------+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner|   2

In [ ]:
df.count()

7907

#Train-test Split

#• Split the data into train and test (80:20). Train the linear regression model using train data and evaluate the model on test data

In [ ]:
#feature extraction
final_data = df.select("features", "selling_price")
final_data.show()

+--------------------+-------------+
|            features|selling_price|
+--------------------+-------------+
|(1997,[0,1982,198...|       450000|
|(1997,[730,1982,1...|       370000|
|(1997,[227,1983,1...|       158000|
|(1997,[1485,1982,...|       225000|
|(1997,[1055,1983,...|       130000|
|(1997,[917,1983,1...|       440000|
|(1997,[291,1985,1...|        96000|
|(1997,[1591,1983,...|        45000|
|(1997,[1169,1982,...|       350000|
|(1997,[147,1982,1...|       200000|
|(1997,[127,1982,1...|       500000|
|(1997,[381,1983,1...|        92000|
|(1997,[21,1982,19...|       280000|
|(1997,[41,1983,19...|       180000|
|(1997,[954,1983,1...|       400000|
|(1997,[75,1982,19...|       778000|
|(1997,[110,1982,1...|       500000|
|(1997,[10,1983,19...|       150000|
|(1997,[234,1982,1...|       680000|
|(1997,[358,1982,1...|       174000|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
train , test = df.randomSplit([0.8,0.2], seed = 123)

In [ ]:
print('Train dataset count:', train.count())
print('Test dataset count:', test.count())

Train dataset count: 6292
Test dataset count: 1615


#Post-split Data Processing

#Before standardization

#Build and Train Model

#Evaluate Model

In [ ]:
# from pyspark.ml.feature import StandardScaler

# # Fit scaler to train dataset
# scaler = StandardScaler().setInputCol('features').setOutputCol('scaled_features')
# scaler_model = scaler.fit(train)

# # Scale train and test features
# train = scaler_model.transform(train)
# test = scaler_model.transform(test)

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='selling_price')
lr_model = lr.fit(train)

In [ ]:
train_predictions = lr_model.transform(train)
test_predictions = lr_model.transform(test)

In [ ]:
pred_test = lr_model.evaluate(test)
pred_test.predictions.show(5)

+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+----------+-------------------+----------+-------------+-----------------+---------------+------------------+----------------+-----------+-------------+--------------------+------------------+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|name_index|           name_vec|fuel_index|     fuel_vec|seller_type_index|seller_type_vec|transmission_index|transmission_vec|owner_index|    owner_vec|            features|        prediction|
+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+----------+-------------------+----------+-------------+-----------------+---------------+------------------+----------------+-----------+-------------+--------------------+------------------+
|Ambassador Grand ...|2008|       122000

#Compute RMSE, MAE and R2 score for the test data.

#RMSE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="selling_price", metricName="rmse")

print("Train_RMSE:", evaluator.evaluate(train_predictions))
print("Test_RMSE:", evaluator.evaluate(test_predictions))

Train_mae: 85665.04015673522
Test_mae: 276134.5252242302


#MAE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="selling_price", metricName="mae")

print("Train_mae:", evaluator.evaluate(train_predictions))
print("Test_mae:", evaluator.evaluate(test_predictions))

Train_mae: 43574.843551442245
Test_mae: 96353.53617145077


#R2 SCORE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="selling_price", metricName="r2")

print("Train_R2:", evaluator.evaluate(train_predictions))
print("Test_R2:", evaluator.evaluate(test_predictions))

Train_R2: 0.988836921152184
Test_R2: 0.8876889222002864


#AFTER STANDARDIZATION

In [ ]:
from pyspark.ml.feature import StandardScaler

# Fit scaler to train dataset
scaler = StandardScaler().setInputCol('features').setOutputCol('scaled_features')
scaler_model = scaler.fit(train)

# Scale train and test features
train = scaler_model.transform(train)
test = scaler_model.transform(test)

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='scaled_features', labelCol='selling_price')
lr_model = lr.fit(train)

In [ ]:
train_predictions = lr_model.transform(train)
test_predictions = lr_model.transform(test)

In [ ]:
pred_test = lr_model.evaluate(test)
pred_test.predictions.show(5)

+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+----------+-------------------+----------+-------------+-----------------+---------------+------------------+----------------+-----------+-------------+--------------------+--------------------+------------------+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|name_index|           name_vec|fuel_index|     fuel_vec|seller_type_index|seller_type_vec|transmission_index|transmission_vec|owner_index|    owner_vec|            features|     scaled_features|        prediction|
+--------------------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+-----+----------+-------------------+----------+-------------+-----------------+---------------+------------------+----------------+-----------+-------------+--------------------+------------------

#RMSE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="selling_price", metricName="rmse")

print("Train_RMSE:", evaluator.evaluate(train_predictions))
print("Test_RMSE:", evaluator.evaluate(test_predictions))

Train_RMSE: 85665.04051589577
Test_RMSE: 276140.2726306039


#MAE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="selling_price", metricName="rmse")

print("Train_MAE:", evaluator.evaluate(train_predictions))
print("Test_MAE:", evaluator.evaluate(test_predictions))

Train_MAE: 85665.04051589577
Test_MAE: 276140.2726306039


#R2

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="selling_price", metricName="r2")

print("Train_R2:", evaluator.evaluate(train_predictions))
print("Test_R2:", evaluator.evaluate(test_predictions))

Train_R2: 0.988836921058579
Test_R2: 0.8876842469129678
